In [62]:
'''
HGU
CSEE
KGH

Fuzzy art + map
'''

'\nHGU\nCSEE\nKGH\n\nFuzzy art + map\n'

In [63]:
from numpy import linalg as LA
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from numpy import linalg as LA

In [122]:
class Fuzzy_ARTMAP():
    def __init__(self, feature_space = 1, threshold = 0.5, w_list = [] , label_list = []):
        print("##model is created##")
        self.M = feature_space
        self.threshold = threshold
        self.w_list = w_list
        self.label_list = label_list
        
    def complement_coding(self, input_vector):
        #normalization_vector(input_vector) 
        complement_vector = [1-i for i in input_vector]
        return complement_vector
    
    def make_input_x(self, input_i):
        return input_i + self.complement_coding(input_i)
       
    def component_wise_min(self, x,y):
        if len(x) != len(y):
            return print("vector length unmatched!")
        zip_list = list(zip(x,y))
        min_list = [min(i,j) for i,j in zip_list]
        return min_list
 
    
    def choice_function(self, x, w):#𝑇_𝑗=|x∧𝐰_𝑗 |+(1−𝛼)(2𝑀−|𝐰_𝑗 |)  (Choice by difference)
        #|𝐱∧𝐰_𝑗 |
        non_zero_a = 0.000001
        x_w = self.component_wise_min(x,w)
        x_w_norm = sum(x_w)
        w_norm = sum(w)
        #𝑇_𝑗=|x∧𝐰_𝑗 |+(1−𝛼)(2𝑀−|𝐰_𝑗 |)  범위가 0~2 인듯??
        T_j = x_w_norm + (1-non_zero_a)*(2*self.M - w_norm)
        return T_j    
    
    #find nearest w at input x
    def Code_competition(self, input_x, label):
        choice_list = [self.choice_function(input_x, w) for w in self.w_list]
        print("choice list:", choice_list)
        #indices = [i for i, x in enumerate(t_list) if x == 1.0] # is the input in any categories?
        #if input belong to any w
        return self.template_matching(input_x, label, choice_list)

        
    def template_matching(self, input_x, label ,choice_list):  
        for idx, index in enumerate(np.flip(np.argsort(choice_list))): # sort the up-down and test each in order
            #|𝑅_𝐽⊕𝐈"|=𝑀−|𝐱∧𝐰_𝐽 |≤𝑀(1−𝜌)
            #print(self.w_list[index])
            x_w =self.component_wise_min(input_x , self.w_list[index])
            # matching template and label
            if self.label_list[index] == label:
                matching_bool = ((self.M - sum(x_w))<=(self.M-self.threshold))
                if matching_bool == True : # if matching is true, extend that w-th boundary 
                    self.w_list[index] = self.component_wise_min(input_x, self.w_list[index]) # template learning
                    print("template learning",  self.w_list )
                    print("Related template:", self.w_list[index]  )
                    print("Related label:", self.label_list[index]  )
                    return self.w_list, self.label_list[index]
                
        # if any boundary is unmatched, make new boundary by input_x
        self.w_list = self.w_list + [input_x] # Category addition
        self.label_list = self.label_list + [label]
        print("Category addition", self.w_list)
        print("additional label:", label)
        return  self.w_list, label
    
    def Train(self, input_i ,label):
        input_x = self.make_input_x(input_i)
        self.w_list, category  = self.Code_competition(input_x, label)
        return category

    
    def Test(self, input_i):
        input_x = self.make_input_x(input_i)
        # select max choice Function value
        choice_list = [self.choice_function(input_x, w) for w in self.w_list]
        print("choice list:", choice_list)
        print("Related Template:", self.w_list[choice_list.index(max(choice_list))] )
        return self.label_list[choice_list.index(max(choice_list))]
        
        
    def info(self):
        print("feature space = ", self.M)
        print("threshold = ", self.threshold)
        print("w_list =", self.w_list)
        print("label_list: = ", self.label_list)
        print("num of categories:", len(self.w_list)) 
        
    def plot_ART(self):
        pass   
        
    def Reset(self):
        self.w_list = []
        self.label_list = []
        return print("Reset the all of templates and labels")
    
    def Save(self, path = os.getcwd()):
        np_label = np.asarray(self.label_list)
        np_w = np.asarray(self.w_list)
        np.save(path+"\\label_info.npy",self.label_list)
        np.save(path+"\\template_info.npy",self.w_list)        
        
    def Load(self, path = os.getcwd()):
        self.label_list = np.load(path+"\\label_info.npy").tolist()
        self.w_list = np.load(path+"\\template_info.npy").tolist()
        

In [123]:
beta = Fuzzy_ARTMAP()

##model is created##


In [124]:
input_1 = [0.1,0.1]

In [96]:
beta.Train(input_1 , 3)

choice list: []
Category addition [[0.1, 0.1, 0.9, 0.9]]
additional label: 3


3

In [97]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.9, 0.9]]
label_list: =  [3]
num of categories: 1


In [98]:
input_2 = [0.3,0.3]

In [99]:
beta.Train(input_2, 2)

choice list: [1.5999999999999999]
Category addition [[0.1, 0.1, 0.9, 0.9], [0.3, 0.3, 0.7, 0.7]]
additional label: 2


2

In [100]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.9, 0.9], [0.3, 0.3, 0.7, 0.7]]
label_list: =  [3, 2]
num of categories: 2


In [101]:
beta.Train(input_2, 3)

choice list: [1.5999999999999999, 2.0]
template learning [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7]]
Related template: [0.1, 0.1, 0.7, 0.7]
Related label: 3


3

In [102]:
input_3 = [0.2,0.2]

In [103]:
beta.Train(input_3, 1)

choice list: [1.9999996, 1.8000000000000003]
Category addition [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8]]
additional label: 1


1

In [104]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8]]
label_list: =  [3, 2, 1]
num of categories: 3


In [105]:
input_4 = [1,1]

In [106]:
beta.Test(input_4)

choice list: [0.5999996000000001, 0.6000000000000002, 0.4]
Related Template: [0.3, 0.3, 0.7, 0.7]


2

In [107]:
beta.Train(input_4,1)

choice list: [0.5999996000000001, 0.6000000000000002, 0.4]
Category addition [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [1, 1, 0, 0]]
additional label: 1


1

In [108]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [1, 1, 0, 0]]
label_list: =  [3, 2, 1, 1]
num of categories: 4


In [109]:
beta.Test(input_4)

choice list: [0.5999996000000001, 0.6000000000000002, 0.4, 2.0]
Related Template: [1, 1, 0, 0]


1

In [110]:
input_5 = [0.9,0.9]

In [111]:
beta.Train(input_5, 1)

choice list: [0.7999996, 0.8000000000000002, 0.6, 1.8]
template learning [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [0.9, 0.9, 0, 0]]
Related template: [0.9, 0.9, 0, 0]
Related label: 1


1

In [112]:
beta.Train(input_4, 5)

choice list: [0.5999996000000001, 0.6000000000000002, 0.4, 1.9999997999999999]
Category addition [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [0.9, 0.9, 0, 0], [1, 1, 0, 0]]
additional label: 5


5

In [113]:
beta.Train([0.95,0.95], 5 )

choice list: [0.6999996000000002, 0.7000000000000003, 0.5000000000000001, 1.9999997999999999, 1.9]
template learning [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [0.9, 0.9, 0, 0], [0.95, 0.95, 0, 0]]
Related template: [0.95, 0.95, 0, 0]
Related label: 5


5

In [114]:
beta.Test([0.94,0.94])

choice list: [0.7199996000000002, 0.7200000000000003, 0.5200000000000001, 1.9999997999999999, 1.9799999]
Related Template: [0.9, 0.9, 0, 0]


1

In [115]:
beta.Test([0.95,0.95])

choice list: [0.6999996000000002, 0.7000000000000003, 0.5000000000000001, 1.9999997999999999, 1.9999999]
Related Template: [0.95, 0.95, 0, 0]


5

In [116]:
beta.Save()

In [117]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [0.9, 0.9, 0, 0], [0.95, 0.95, 0, 0]]
label_list: =  [3, 2, 1, 1, 5]
num of categories: 5


In [118]:
beta.Reset()

Reset the all of templates and labels


In [119]:
beta.info()

feature space =  1
threshold =  0.5
w_list = []
label_list: =  []
num of categories: 0


In [120]:
beta.Load()

In [121]:
beta.info()

feature space =  1
threshold =  0.5
w_list = [[0.1, 0.1, 0.7, 0.7], [0.3, 0.3, 0.7, 0.7], [0.2, 0.2, 0.8, 0.8], [0.9, 0.9, 0.0, 0.0], [0.95, 0.95, 0.0, 0.0]]
label_list: =  [3, 2, 1, 1, 5]
num of categories: 5
